In [ ]:
!pip install contractions

In [41]:
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('words')

In [42]:
import re
import pickle
import numpy as np
import pandas as pd
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
df_1 = pd.read_csv("../Dataset/csv/train.csv", header=0, index_col=0)
df_1 = df_1.drop(['title','author'], axis = 1)
df_1.dropna(inplace = True)
#print(df_1.isnull().sum(axis = 0))

#Expanding Contraction, Lower Case and Word Splitting
df_1['text'] = df_1['text'].apply(lambda x: [contractions.fix(word, slang=False).lower() for word in x.split()])

#Removing Punctuations
df_1['text'] = df_1['text'].apply(lambda x: [re.sub(r'[^\w\s]','', word)  for word in x])

#Removing Stop Words
stop_words = set(stopwords.words('english'))
df_1['text'] = df_1['text'].apply(lambda x: [word for word in x if word not in stop_words])

#Removing Special Charecter and Numbers
df_1['text'] = df_1['text'].apply(lambda x: [word for word in x if re.search("[@_!#$%^&*()<>?/|}{~:0-9]", word) == None])

#Removing Non-English Words
english_words = set(nltk.corpus.words.words())
df_1['text'] = df_1['text'].apply(lambda x: [word for word in x if word in english_words])

#Concating Words back to Sentence
df_1['text'] = df_1['text'].apply(lambda x: ' '.join(x))

print(df_1.head(3))

                                                 text  label
id                                                          
0   house aide even see letter subscribe stump for...      1
1   ever get feeling life roundabout rather straig...      0
2   truth might get fired tension intelligence pol...      1


In [47]:
df_2 = pd.read_csv("../Dataset/csv/test.csv", header=0, index_col=0)
df_2 = df_2.drop(['title','author'], axis = 1)
df_2.dropna(inplace = True)
#print(df_2.isnull().sum(axis = 0))

#Expanding Contraction, Lower Case and Word Splitting
df_2['text'] = df_2['text'].apply(lambda x: [contractions.fix(word, slang=False).lower() for word in x.split()])

#Removing Punctuations
df_2['text'] = df_2['text'].apply(lambda x: [re.sub(r'[^\w\s]','', word)  for word in x])

#Removing Stop Words
stop_words = set(stopwords.words('english'))
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if word not in stop_words])

#Removing Special Charecter and Numbers
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if re.search("[@_!#$%^&*()<>?/|}{~:0-9]", word) == None])

#Removing Non-English Words
english_words = set(nltk.corpus.words.words())
df_2['text'] = df_2['text'].apply(lambda x: [word for word in x if word in english_words])

#Concating Words back to Sentence
df_2['text'] = df_2['text'].apply(lambda x: ' '.join(x))

#Transforming Labels
df_2['label'] = df_2['label'].apply(lambda x: int(x))

print(df_2.head(3))

                                                    text  label
id                                                             
20800  alto political process silicon valley leapt fr...      0
20801  ready strike near source source attack aircraf...      1
20802  native vow stay winter file lawsuit police amn...      0


In [48]:
tfidfVectorizer = TfidfVectorizer(use_idf=True,stop_words='english')
tfidfVectorizer.fit(df_1['text'])  
tfidfVectorizer.fit(df_2['text']) 
#print(tfidfVectorizer.get_feature_names())

trainTfidfVector = tfidfVectorizer.transform(df_1['text']) 
testTfidfVector = tfidfVectorizer.transform(df_2['text']) 

pickle.dump(tfidfVectorizer, open("../Dataset/tfidf_vectorizer-1.pickle", "wb"))

pickle.dump(trainTfidfVector, open("../Dataset/tfidf_train-1.pickle", "wb"))
pickle.dump(df_1['label'], open("../Dataset/label_train-1.pickle", "wb"))

pickle.dump(testTfidfVector, open("../Dataset/tfidf_test-1.pickle", "wb"))
pickle.dump(df_2['label'], open("../Dataset/label_test-1.pickle", "wb"))


In [49]:
tfidfVectorizer = pickle.load(open("../Dataset/tfidf_vectorizer-1.pickle", "rb"))
#print(tfidfVectorizer.get_feature_names())

trainTfidfVector = pickle.load(open("../Dataset/tfidf_train-1.pickle", "rb"))
df_train = pd.DataFrame(data = trainTfidfVector.toarray(),columns = tfidfVectorizer.get_feature_names())
#print(df_train)

testTfidfVector = pickle.load(open("../Dataset/tfidf_test-1.pickle", "rb"))
df_test = pd.DataFrame(data = testTfidfVector.toarray(),columns = tfidfVectorizer.get_feature_names())
#print(df_test)

trainLabels = pickle.load(open("../Dataset/label_train-1.pickle", "rb"))
#print(trainLabels)

testLabels = pickle.load(open("../Dataset/label_test-1.pickle", "rb"))
#print(testLabels)